# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

**Описание данных**

Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц. 

Известно:
- сalls — количество звонков,
- minutes — суммарная длительность звонков в минутах,
- messages — количество sms-сообщений,
- mb_used — израсходованный интернет-трафик в Мб,
- is_ultra — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## Откройте и изучите файл

In [ ]:
import pandas as pd
import pandas_profiling
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [3]:
# открытие файла с данными
df = pd.read_csv('/datasets/users_behavior.csv')
print(df.head())

   calls  minutes  messages   mb_used  is_ultra
0   40.0   311.90      83.0  19915.42         0
1   85.0   516.75      56.0  22696.96         0
2   77.0   467.66      86.0  21060.45         0
3  106.0   745.53      81.0   8437.39         1
4   66.0   418.74       1.0  14502.75         0


In [4]:
#загрузка общей информации
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [5]:
#проверка пропусков
df.isna().sum()

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

In [6]:
display(df.describe().T)

,count,mean,std,min,25%,50%,75%,max
calls,3214.0,63.038892,33.236368,0.0,40.0000,62.000,82.0000,244.00
minutes,3214.0,438.208787,234.569872,0.0,274.5750,430.600,571.9275,1632.06
messages,3214.0,38.281269,36.148326,0.0,9.0000,30.000,57.0000,224.00
mb_used,3214.0,17207.673836,7570.968246,0.0,12491.9025,16943.235,21424.7000,49745.73
is_ultra,3214.0,0.306472,0.461100,0.0,0.0000,0.000,1.0000,1.00


Данные присутствуют в полном обьеме, пропусков нет.Тип данных - корректный

## Разбейте данные на выборки

Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Перед делением на выборки выделим целевой признак(target) и остальные признаки(features)

In [7]:
features = df.drop(['is_ultra'],axis=1)
target = df['is_ultra']
#проверка
display(features.columns.to_list())
display(target.head())

['calls', 'minutes', 'messages', 'mb_used']

0    0
1    0
2    0
3    1
4    0
Name: is_ultra, dtype: int64

Разделение исходных данных на обучающую, валидационную и тестовую выборки в пропорции 60%, 20%и 20%
Для начала разделим данные на обучающую и валидационную (60% и 40%), и потом валиадаионную еще на две равные части(валидационную и тестовую). Что в итоге и образуют данне с распределением 60%, 20%, 20% 

In [8]:
#разбивка на обучающую (60%) и валидационную (40%)
features_train, features_val, target_train, target_val = train_test_split(features,
                                                                          target, 
                                                                          test_size=0.4,
                                                                          random_state=12345)

In [9]:
#разбивка пополам валидационной на тестовую и валидационную ( по 20%)
features_valid, features_test, target_valid, target_test = train_test_split(features_val,
                                                                            target_val,
                                                                            test_size=0.5,
                                                                            random_state=12345)

Получилось три выборки: featires_train (60%), features_valid (20%) ,features_test (20%)

In [17]:
print('размер обучающей выборки:',features_train.shape)
print('размер валидационной выборки:', features_valid.shape)
print('размер тестовой выборки:', features_test.shape)
print(target_train.shape)
print(target_valid.shape)
print(target_test.shape)

размер обучающей выборки: (1928, 4)
размер валидационной выборки: (643, 4)
размер тестовой выборки: (643, 4)
(1928,)
(643,)
(643,)


## Исследуйте модели

In [91]:
#исследуем модель "решающего дерева" 
best_model = None
best_result = 0
for depth in range(1,6):
    model = DecisionTreeClassifier(random_state=12345, max_depth = depth)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    result = accuracy_score(target_valid, predictions_valid)
    if result > best_result:
        best_model = model
        best_result = result
print('Accuracy лучшей модели:',best_result)
print("Глубина дерева лучшей модели:", depth)
print("Лучшая модель:",model)
    

Accuracy лучшей модели: 0.7853810264385692
Глубина дерева лучшей модели: 5
Лучшая модель: DecisionTreeClassifier(max_depth=5, random_state=12345)


In [25]:
# изучение модели при вводе дополнительно гиперпараметра max_features
model = DecisionTreeClassifier(random_state=12345, max_depth = 5, max_features =3)
model.fit(features_train, target_train)
predictions_valid = model.predict(features_valid)
result = accuracy_score(target_valid, predictions_valid)
print('Accuracy лучшей модели:',result)

Accuracy лучшей модели: 0.7931570762052877


при вводе дополнительного гиперпараметра max_features( колиество признаков, необходимых одного ветвления) Accuracy показывает более интересыый вариант: 
- при значении max_features =2, Accuracy= 0.78849; 
- при значении max_features =3, Accuracy= 0.793157
- при значении max_features =4, Accuracy= 0.77916 уже ниже, оставим на значении 3

In [27]:
#вввод дополнительного гиперпараметра criterion = 'gini'
model = DecisionTreeClassifier(random_state=12345, max_depth = 5, max_features =3, criterion = 'gini')
model.fit(features_train, target_train)
predictions_valid = model.predict(features_valid)
result = accuracy_score(target_valid, predictions_valid)
print('Accuracy лучшей модели:',result)

Accuracy лучшей модели: 0.7931570762052877


Гиперпараметр criterion = 'gini' (показывает меру сходства двух наборов данных) в данном случае общее значение Accuracy не меняет

In [92]:
#проверка всез результатов 
for depth in range(1,6):
    model = DecisionTreeClassifier(random_state=12345, max_depth = depth)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    result = accuracy_score(target_valid, predictions_valid)
    print('max_depth=',depth, ":", end ='' )
    print(result)
   
    

max_depth= 1 :0.7542768273716952
max_depth= 2 :0.7822706065318819
max_depth= 3 :0.7853810264385692
max_depth= 4 :0.7791601866251944
max_depth= 5 :0.7791601866251944


In [98]:
#исследование модели "случайный лес"
best_model = None
best_result = 0
best_est = 0
best_depth = 0
for est in range(1,51,10):
    for depth in range (1,11):
        model = RandomForestClassifier(random_state=12345, n_estimators = est, max_depth = depth)
        model.fit(features_train, target_train)
        predictions_valid = model.predict(features_valid)
        result = accuracy_score(target_valid, predictions_valid)
        if result > best_result:
            best_model = model
            best_result = result
            best_est = est
            best_depth = depth
print('Accurancy лучшей модели:',best_result)
print('Количество деревьев лучшей модели:', est )
print ('Глубина дерева лучшей модели:', depth )
print('Лучшая модель:', best_model)
    

Accurancy лучшей модели: 0.807153965785381
Количество деревьев лучшей модели: 41
Глубина дерева лучшей модели: 10
Лучшая модель: RandomForestClassifier(max_depth=8, n_estimators=41, random_state=12345)


In [97]:
#проверка всех значений

for est in range(1,51,10):
    for depth in range (1,11):
        model = RandomForestClassifier(random_state=12345, n_estimators = est, max_depth = depth)
        model.fit(features_train, target_train)
        predictions_valid = model.predict(features_valid)
        result = accuracy_score(target_valid, predictions_valid)
        print('max_est=',est, ":", 'max_depth=',depth, ":", end ='' )
        print(result)

max_est= 1 : max_depth= 1 :0.7542768273716952
max_est= 1 : max_depth= 2 :0.7853810264385692
max_est= 1 : max_depth= 3 :0.7853810264385692
max_est= 1 : max_depth= 4 :0.7744945567651633
max_est= 1 : max_depth= 5 :0.776049766718507
max_est= 1 : max_depth= 6 :0.7853810264385692
max_est= 1 : max_depth= 7 :0.7776049766718507
max_est= 1 : max_depth= 8 :0.7776049766718507
max_est= 1 : max_depth= 9 :0.7791601866251944
max_est= 1 : max_depth= 10 :0.7729393468118196
max_est= 11 : max_depth= 1 :0.7542768273716952
max_est= 11 : max_depth= 2 :0.7853810264385692
max_est= 11 : max_depth= 3 :0.7838258164852255
max_est= 11 : max_depth= 4 :0.7884914463452566
max_est= 11 : max_depth= 5 :0.7853810264385692
max_est= 11 : max_depth= 6 :0.8009331259720062
max_est= 11 : max_depth= 7 :0.7962674961119751
max_est= 11 : max_depth= 8 :0.7993779160186625
max_est= 11 : max_depth= 9 :0.7853810264385692
max_est= 11 : max_depth= 10 :0.7916018662519441
max_est= 21 : max_depth= 1 :0.7636080870917574
max_est= 21 : max_dept

In [96]:
#изучение модели логистической регрессии

model = LogisticRegression(random_state =12345, solver ='lbfgs', max_iter=1000)
model.fit(features_train, target_train)
predictions_valid = model.predict(features_valid)
result = accuracy_score(target_valid, predictions_valid)
print('Accurancy модели логической регресии:', result)

Accurancy модели логической регресии: 0.7107309486780715


In [33]:
#проверка всез значений
for iter in range (100,1001,100):
    model = LogisticRegression(random_state =12345, solver ='lbfgs', max_iter=1000)
    model.fit(features_train, target_train)
    predictions_valid = model.predict(features_valid)
    result = accuracy_score(target_valid, predictions_valid)
    print(f"max_iter = {iter} : {result}")
   

max_iter = 100 : 0.7107309486780715
max_iter = 200 : 0.7107309486780715
max_iter = 300 : 0.7107309486780715
max_iter = 400 : 0.7107309486780715
max_iter = 500 : 0.7107309486780715
max_iter = 600 : 0.7107309486780715
max_iter = 700 : 0.7107309486780715
max_iter = 800 : 0.7107309486780715
max_iter = 900 : 0.7107309486780715
max_iter = 1000 : 0.7107309486780715


**Вывод:**
При сравнении всех моделей получились следующие значения:

 1. Лучшая модель 'дерева решений' имеет долю правильных ответов(accurancy): 0.7853810264385692, при значении глубины дерева (depth =5)
 2. Лучшая модель 'случайного леса' имеет долю правильных ответов (accurancy): 0.80715396578538, при значении глубины дерева (dept=10, количества деревьев (n_estimators = 41) 
 3. Accurancy модели логической регресии: 0.7107309486780715

Наилучшая модель в данном случае - модель случайного леса, в этой модели самый высокий уровень доли правильных ответов(accurany)

## Проверьте модель на тестовой выборке

In [37]:
#проверка моделей на тестовой выборке:
#1. Лучшая модель  дерева решений (depth =5)
model = DecisionTreeClassifier(random_state=12345, max_depth = 5)
model.fit(features_train, target_train)
predictions_test = model.predict(features_test)
result1 = accuracy_score(target_test, predictions_test)
print("Accurancy лучшей модели дерева решений на тестовой выборке:", result1 )



Accurancy лучшей модели дерева решений на тестовой выборке: 0.7838258164852255


In [103]:
# 2.Лучшая модель "случайного леса" (ept=10, n_estimators = 41)
model = RandomForestClassifier(random_state=12345, n_estimators = 41, max_depth = 10)
model.fit(features_train, target_train)
predictions_test = model.predict(features_test)
result2 = accuracy_score(target_test, predictions_test)
print("Accurancy лучшей модели случайного леса на тестовой выборке:", result2)


Accurancy лучшей модели случайного леса на тестовой выборке: 0.7978227060653188


In [109]:
#3. Модель логистической регрессии
model = LogisticRegression(random_state =12345, solver ='lbfgs', max_iter=1000)
model.fit(features_train, target_train)
predictions_test = model.predict(features_test)
result3 = accuracy_score(target_test, predictions_test)
print("Accurancy модели логистической регрессии на тестовой выборке:", result3)

Accurancy модели логистической регрессии на тестовой выборке: 0.6842923794712286


**Вывод:** 
Наилучший результат показала модель случайного леса. И На валидационной выборке и на тестовой лоля правильных ответов выше всего.
Доля правильныз ответов на валидационной выборке составило - 0.807, на тестовой выборке - 0.797.

## Проверьте модели на адекватность

In [39]:
#проверка балансов классов выборки (целевой признак обучайющей выборки)
target_train.value_counts(normalize = 1)

0    0.692427
1    0.307573
Name: is_ultra, dtype: float64

In [40]:
#проверка балансов классов выборки (целевой признак валидационной выборки)
target_valid.value_counts(normalize = 1)

0    0.706065
1    0.293935
Name: is_ultra, dtype: float64

У обучающией и валидационной выборок наблюдается заметный дисбаланс класов. Ответы 0 ≈70% , 1 ≈ 30%
Можно ожидать, что результаты предсказаний моделей будут сильно склоняться к одному варианту ответов.

In [42]:
#созданиие константной модели. За константу примем пример 0, так как это значение встречается 70% случаев
target_predict_constant = pd.Series([0] * len(target_test))
# расчет accuracy  констатной модели
print(f'accuracy_score константной модели: {accuracy_score(target_test, target_predict_constant)}')

accuracy_score константной модели: 0.6842923794712286


Accuracy дерево решений 0.783; случайный лес 0.797; логистическая регрессия 0.684.
Accuracy константной модели: 0.796
Случайный лес показал результат выше констентной модели, в связи с чем, только ее можем признать адекватной